In [1]:
import os
import csv 
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import keras
from keras.utils import np_utils 
from keras.models import Sequential
from keras.models import model_from_json
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, RMSprop
from keras.regularizers import l1, l2,l1l2
import matplotlib.pyplot as plt

/usr/local/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)
Using Theano backend.
/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [6]:
class Data:
	'Class for data object'
	
	def __init__(self,trainFile,trainLabelFile,valRatio):
		self.trainFile = trainFile;
		self.trainLabelFile = trainLabelFile;
# 		self.testFile = testFile;
		self.valRatio = valRatio;
		
	def inputPreprocess(self):
		trainData = pd.read_csv(self.trainFile);
		trainLabel = pd.read_table(self.trainLabelFile,header = None);
		# The number of input features
		
		trainData = trainData.replace(-1,np.nan);
		trainData = trainData.fillna(trainData.mean())
		print trainData.mean()
		self.features = trainData.shape[1];
		print self.features
		trainData = trainData.as_matrix();
		trainLabel = trainLabel.as_matrix();
		# [self.trainInputs, self.valInputs, self.trainTargets, self.valTargets] = train_test_split(trainData,trainLabel ,test_size = self.valRatio)	
# 		

		self.trainInputs = trainData;
		self.trainTargets = trainLabel;
		print self.trainInputs
		self.trainMean = np.mean(self.trainInputs,axis=0)
		tnStd = np.std(self.trainInputs,axis=0,dtype = np.float32)
		self.trainStd = np.array(list(tnStd))
		self.trainInputs = z_score_inputs(self.trainInputs,self.trainMean,self.trainStd);
# 		self.valInputs = z_score_inputs(self.valInputs,self.trainMean,self.trainStd);
		print self.trainInputs
		
		
def z_score_inputs(Inputs, Mean, StdDev):
	""" 
		Pre-process inputs by making it mean zero and unit standard deviation
	"""
# 	for i in range(StdDev.size):
		# if (StdDev[i] < 0.02):
# 			StdDev[i] = 0.02
	
	Inputs = np.divide((Inputs-Mean),StdDev)
	return Inputs
	
	
class Model:

	def __init__(self,neurons,activation,optimizer,errFunc = 'mse',epochs=200, wd= 0.01,dropout = 0.0):
		self.neurons = neurons;
		self.errFunc = errFunc;
		self.activation = activation;
		self.optimizer = optimizer;
		self.epochs = epochs;
		self.wd = wd;
		self.dropout = dropout;
		
		
	def modelTrainValidate(self,data):
		
		# Callback used to track crossentropy loss during training
	
		class LossHistory(keras.callbacks.Callback):
			def on_train_begin(self,logs={}):
				self.losses = []
		
			def on_batch_end(self, batch, logs={}):
				self.losses.append(logs.get('loss'))
		
		# Build the NN model
		
		model = Sequential();
		model.add(Dense(self.neurons,input_dim = data.features,init = 'glorot_uniform', W_regularizer = l2(self.wd),activation=self.activation[0]));
		model.add(Dropout(self.dropout));
		model.add(Dense(1,activation = self.activation[1]));
		model.compile(loss = self.errFunc, optimizer = self.optimizer);
		
		history = LossHistory()
		
		# Train the model
		
		model.fit(data.trainInputs, data.trainTargets, nb_epoch = self.epochs, batch_size = 32, callbacks = [history]);
		predictions = model.predict(data.trainInputs);
		
		print '*****PREDICTIONS*****'
		
		print predictions
		plt.figure()
		plt.plot(history.losses)
		plt.show()
		
		
def main():
	
	activation = np.array(['relu','relu'],dtype=object)
	
	error_function = 'mse'
	optimizer_model = 'sgd'
	
	data1 = Data('./train.csv','./train_labels.txt',0.30);
	
	data1.inputPreprocess();
	
	model1 = Model(150,activation,optimizer_model);
	
	model1.modelTrainValidate(data1);
	
	


In [ ]:
if __name__ == "__main__":
	main()
	


SE.PRM.TENR             8.462768e+01
SE.PRM.TENR.FE          8.157562e+01
SE.PRM.TENR.MA          8.540648e+01
NY.ADJ.NNTY.KD.ZG       3.851237e+00
NY.ADJ.NNTY.KD          1.621273e+12
NY.ADJ.NNTY.CD          9.929433e+11
NY.ADJ.NNTY.PC.KD.ZG    2.163827e+00
NY.ADJ.NNTY.PC.KD       7.589344e+03
NY.ADJ.NNTY.PC.CD       5.100576e+03
NY.ADJ.DCO2.GN.ZS       5.851388e-01
NY.ADJ.DCO2.CD          7.534420e+08
NY.ADJ.DKAP.GN.ZS       1.032384e+01
NY.ADJ.DKAP.CD          2.605851e+10
NY.ADJ.AEDU.GN.ZS       3.979355e+00
NY.ADJ.AEDU.CD          7.909355e+09
NY.ADJ.DNGY.GN.ZS       3.739394e+00
NY.ADJ.DNGY.CD          2.847644e+09
NY.ADJ.DMIN.GN.ZS       5.832514e-01
NY.ADJ.DMIN.CD          3.627539e+08
NY.ADJ.DRES.GN.ZS       6.098861e+00
NY.ADJ.DFOR.GN.ZS       1.423187e+00
NY.ADJ.DFOR.CD          2.363728e+08
SP.ADO.TFRT             7.711841e+01
SP.POP.DPND             7.262675e+01
SP.POP.DPND.OL          1.003267e+01
SP.POP.DPND.YG          6.250853e+01
AG.LND.AGRI.ZS          3.764228e+01
A